<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/tensorflow/tensorflow_roberta_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# read to use TPU https://blog.tensorflow.org/2019/01/keras-on-tpus-in-colab.html

In [17]:
# To avoid notebook get disconnected
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)
"""

"""
caffeinate
"""

'\ncaffeinate\n'

In [18]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sat May  2 18:09:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    31W / 250W |   1413MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------

In [19]:
!pip install transformers
import os
import pandas as pd, numpy as np
import time, re, math
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version',tf.__version__)

TF version 2.2.0-rc3


In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
ROBERTA_PATH = INPUT_PATH + 'tf-roberta/'

VER='v8'
DISPLAY=1 # USE display=1 FOR INTERACTIVE

DEV_MODE = False # if True only looks at a subset of data
MAX_SUM_PROB = False 

In [0]:
def remove_html(text):
    text = re.sub("&quot;", '"', text)
    text = re.sub("&gt;", ">", text)
    text = re.sub("&lt;", "<", text)
    text = re.sub("&le;", "≤", text)
    text = re.sub("&ge;", "≥", text)
    text = re.sub("&amp;", "&", text)
    return text

def add_html(text):
    text = re.sub('"', "&quot;",  text)
    text = re.sub(">", "&gt;", text)
    text = re.sub("<", "&lt;", text)
    text = re.sub("≤", "&le;", text)
    text = re.sub("≥", "&ge;", text)
    text = re.sub("&", "&amp;", text)   
    return text
#TODO: find examples and apply!!

def print_to_file(text):
  try:
    f = open(CHECKPOINT_PATH + VER + "_logs.txt", 'a')
  except:
    f = open(CHECKPOINT_PATH + VER + "_logs.txt", 'w')

  print(text)
  f.write(time.strftime("%Y%m%d-%H%M%S"))
  f.write("  -- ")
  f.write(text)
  f.write("\n")

In [0]:
def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0

In [0]:
MAX_LEN = 120 #96 #TODO: make this value high enough that this doesn't cause an issue in the final setting
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file= ROBERTA_PATH +'vocab-roberta-base.json', 
    merges_file= ROBERTA_PATH +'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}

# READ DATA

In [24]:
def read_train():
    train=pd.read_csv(DATA_PATH + 'train.csv')
    train['text']=train['text'].astype(str)
    train['selected_text']=train['selected_text'].astype(str)

    return train

def read_test():
    test=pd.read_csv(DATA_PATH + 'test.csv')
    test['text']=test['text'].astype(str)

    return test

if (DEV_MODE == True):
  train_df = read_train().sample(frac=.05, random_state=1).reset_index()
  test_df = read_test().sample(frac=.05, random_state=1).reset_index()
else:
  train_df = read_train()
  test_df = read_test()


print_to_file("train_df shape:" + str(train_df.shape))
print_to_file("test_df shape:" + str(test_df.shape))

train_df shape:(27481, 4)
test_df shape:(3534, 3)


# PREPROCESS

In [0]:
ct = train_df.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train_df.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train_df.loc[k,'text'].split())
    text2 = " ".join(train_df.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train_df.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [0]:
ct = test_df.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test_df.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test_df.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test_df.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

# Model

In [0]:
def scheduler(epoch):
    return 3e-5 * 0.2**epoch

def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(ROBERTA_PATH + 'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(ROBERTA_PATH + 'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    
    x1 = tf.keras.layers.Dropout(0.3)(x[0]) 
    x1 = tf.keras.layers.Conv1D(128, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.3)(x[0]) 
    x2 = tf.keras.layers.Conv1D(128, 2, padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(64, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])

    return model

In [28]:
model = build_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 120)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 120)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 120)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model_1 (TFRobertaMo ((None, 120, 768), ( 124645632   input_4[0][0]                    
____________________________________________________________________________________________

# Train

In [0]:
# tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver) 

In [0]:
n_splits = 5

jac = []
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train_df.sentiment.values)):

    print_to_file('#'*25)
    print_to_file('### FOLD %i'%(fold+1))
    print_to_file('#'*25)


    K.clear_session()
    if not os.path.exists(CHECKPOINT_PATH + '%s-roberta-%i.h5'%(VER,fold)):
        # with strategy.scope():
        model = build_model()
        model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5),
            loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=.25)
            )
        
        reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

        sv = tf.keras.callbacks.ModelCheckpoint(
            CHECKPOINT_PATH + '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
            save_weights_only=True, mode='auto', save_freq='epoch')
            
        X = [input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]]
        Y = [start_tokens[idxT,], end_tokens[idxT,]]
        X_valid = [input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]]
        Y_valid = [start_tokens[idxV,], end_tokens[idxV,]]
        hist = model.fit(
            X,
            Y,
            epochs=3,
            batch_size=8,
            verbose=DISPLAY,
            callbacks=[sv, reduce_lr],
            validation_data=(X_valid, Y_valid)
            )
    else:
      print_to_file("training Fold %d is skipped!"%(fold+1))

    print_to_file('Loading model...')
    model.load_weights(CHECKPOINT_PATH + '%s-roberta-%i.h5'%(VER,fold))
    
    print_to_file('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])

        if (a>b) and MAX_SUM_PROB: 
            max_len = len(oof_start[k,])
            a = np.tile(oof_start[k,], (max_len, 1))
            b = np.tile(oof_end[k,], (max_len, 1))
            c = np.tril(a + b.T, k=0).T
            c[c == 0] = -1000
            a = np.unravel_index(c.argmax(), c.shape)[0]
            b = np.unravel_index(c.argmax(), c.shape)[1]

        if (a>b):
            st = train_df.loc[k,'text'] 
        else:
            text1 = " "+" ".join(train_df.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-1:b])
        
        train_df.loc[k,'selected_text_predicted'] = st
        train_df.loc[k,'jaccard'] = jaccard(train_df.loc[k,'selected_text_predicted'],train_df.loc[k,'selected_text'])
        if train_df.loc[k,'source'] == "competition":
            all.append(train_df.loc[k,'jaccard'])

    jac.append(np.mean(all))
    print_to_file('>>>> FOLD %i Jaccard = '%(fold+1) + str(np.mean(all)))
    print()


print_to_file('Overall averrage of Jaccards = ' + str(np.mean(jac)))

#########################
### FOLD 1
#########################
Epoch 1/3
 977/2748 [=========>....................] - ETA: 3:59 - loss: 1.2630 - activation_loss: 0.6316 - activation_1_loss: 0.6314

In [0]:
# #########################
# Epoch 1/3
# WARNING:tensorflow:Gradients do not exist for variables ['tf_roberta_model/roberta/pooler/dense/kernel:0', 'tf_roberta_model/roberta/pooler/dense/bias:0'] when minimizing the loss.
# WARNING:tensorflow:Gradients do not exist for variables ['tf_roberta_model/roberta/pooler/dense/kernel:0', 'tf_roberta_model/roberta/pooler/dense/bias:0'] when minimizing the loss.
# WARNING:tensorflow:Gradients do not exist for variables ['tf_roberta_model/roberta/pooler/dense/kernel:0', 'tf_roberta_model/roberta/pooler/dense/bias:0'] when minimizing the loss.
# WARNING:tensorflow:Gradients do not exist for variables ['tf_roberta_model/roberta/pooler/dense/kernel:0', 'tf_roberta_model/roberta/pooler/dense/bias:0'] when minimizing the loss.
# 2748/2748 [==============================] - ETA: 0s - loss: 0.0325 - activation_loss: 0.0165 - activation_1_loss: 0.0160
# Epoch 00001: val_loss improved from inf to 0.02835, saving model to /content/drive/My Drive/input/model_checkpoint/roberta/v6-roberta-0.h5
# 2748/2748 [==============================] - 351s 128ms/step - loss: 0.0325 - activation_loss: 0.0165 - activation_1_loss: 0.0160 - val_loss: 0.0283 - val_activation_loss: 0.0145 - val_activation_1_loss: 0.0138 - lr: 3.0000e-05
# Epoch 2/3
# 2748/2748 [==============================] - ETA: 0s - loss: 0.0256 - activation_loss: 0.0131 - activation_1_loss: 0.0125
# Epoch 00002: val_loss improved from 0.02835 to 0.02735, saving model to /content/drive/My Drive/input/model_checkpoint/roberta/v6-roberta-0.h5
# 2748/2748 [==============================] - 349s 127ms/step - loss: 0.0256 - activation_loss: 0.0131 - activation_1_loss: 0.0125 - val_loss: 0.0273 - val_activation_loss: 0.0140 - val_activation_1_loss: 0.0134 - lr: 6.0000e-06
# Epoch 3/3
# 2748/2748 [==============================] - ETA: 0s - loss: 0.0240 - activation_loss: 0.0122 - activation_1_loss: 0.0117
# Epoch 00003: val_loss did not improve from 0.02735
# 2748/2748 [==============================] - 342s 125ms/step - loss: 0.0240 - activation_loss: 0.0122 - activation_1_loss: 0.0117 - val_loss: 0.0275 - val_activation_loss: 0.0140 - val_activation_1_loss: 0.0136 - lr: 1.2000e-06
# Loading model...
# Predicting OOF...
# 172/172 [==============================] - 23s 135ms/step
# >>>> FOLD 1 Jaccard = 0.7045174031165262


# Inference

In [0]:
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))
DISPLAY=1
for i in range(5):
    print_to_file('#'*25)
    print_to_file('### MODEL %i'%(i+1))
    print_to_file('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights(CHECKPOINT_PATH + '%s-roberta-%i.h5'%(VER,i))

    print_to_file('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/n_splits
    preds_end += preds[1]/n_splits


# Prepare Results

In [0]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])

    if (a>b) and MAX_SUM_PROB: 
        max_len = len(oof_start[k,])
        a = np.tile(oof_start[k,], (max_len, 1))
        b = np.tile(oof_end[k,], (max_len, 1))
        c = np.tril(a + b.T, k=0).T
        c[c == 0] = -1000
        a = np.unravel_index(c.argmax(), c.shape)[0]
        b = np.unravel_index(c.argmax(), c.shape)[1]

    if (a>b): 
        st = test_df.loc[k,'text']
    else:
        text1 = " "+" ".join(test_df.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-1:b])
    all.append(st)

test_df['selected_text'] = all
test_df['selected_text'] = test_df['selected_text'].apply(add_html)

test_df['selected_text'] = test_df['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
test_df['selected_text'] = test_df['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
test_df['selected_text'] = test_df['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)

test_df[['textID','selected_text']].to_csv('submission.csv',index=False)